### Load and preprocessing images  

In [1]:
import os
import cv2
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# The address of the folder where the images are located
path = "C:\\Users\\Amin\\Desktop\\Mask"

images = []
labels = []
image_list = os.listdir(path)
image_size = 64
for image_name in image_list: 
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (image_size,image_size)) 
    image = image.astype('float32')         
    image = image / 255.0                           
    images.append(image)
    labels.append(image_name[-5])